# Perform variant QC

In this notebook, we perform variant quality control using [PLINK2](https://www.cog-genomics.org/plink/2.0/).

# Setup 

<div class="alert alert-block alert-warning">
    <b>Cloud Environment</b>: This notebook was written for use on the All of Us Workbench.
    <ul>
        <li>Use compute type 'Standard VM' with sufficient CPU and RAM (e.g. start with 8 CPUs and 30 GB RAM, increase if needed).</li>
        <li>This notebook can take a while to run. Recommend that it is run in the background via <kbd>run_notebook_in_the_background</kbd>.</li>
    </ul>
</div>

In [ ]:
from datetime import datetime
import os
import time

## Setup plink2

https://www.cog-genomics.org/plink/2.0/

In [ ]:
%%bash

##### plink 2 install
PLINK_VERSION=2.3.Alpha
PLINK_ZIP_PATH=/tmp/plink-$PLINK_VERSION.zip
curl -L -o $PLINK_ZIP_PATH https://s3.amazonaws.com/plink2-assets/alpha2/plink2_linux_x86_64.zip
mkdir -p /tmp/plink2/
unzip -o $PLINK_ZIP_PATH -d /tmp/plink2/

In [ ]:
!/tmp/plink2/plink2 --version # --help

## Define constants

In [ ]:
# Papermill parameters. See https://papermill.readthedocs.io/en/latest/usage-parameterize.html

#---[ Inputs ]---
# The BGEN file created via write_bgen.ipynb.
REMOTE_MERGED_BGEN = 'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210906/ukb-aou-alpha2-chr1-chr22.bgen'
REMOTE_MERGED_BGEN_SAMPLE = 'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/merged/20210906/ukb-aou-alpha2-chr1-chr22.sample'
# This CSV was created via notebook 7_pooled_lipids_gwas_phenotype.ipynb.
REMOTE_PHENOTYPES = 'gs://fc-secure-fd6786bf-6c28-4f33-ac30-3860fbeee5bb/data/pooled/phenotypes/20211224/aou_alpha2_ukb_pooled_lipids_phenotype.tsv'

#---[ Outputs ]---
# Create a timestamp for a folder of results generated today.
DATESTAMP = time.strftime('%Y%m%d')
OUTPUT_FILENAME_PREFIX = 'aou_alpha3_ukb_lipids'
OUTPUT_FOLDER = f'{os.getenv("WORKSPACE_BUCKET")}/data/pooled/variant-qc/{DATESTAMP}/'

In [ ]:
LOCAL_MERGED_BGEN = os.path.basename(REMOTE_MERGED_BGEN)
LOCAL_MERGED_BGEN_SAMPLE = os.path.basename(REMOTE_MERGED_BGEN_SAMPLE)
LOCAL_PHENOTYPES = os.path.basename(REMOTE_PHENOTYPES)

## Copy data locally

In [ ]:
!gsutil cp -n {REMOTE_MERGED_BGEN} {REMOTE_MERGED_BGEN_SAMPLE} .

In [ ]:
!gsutil cp {REMOTE_PHENOTYPES} .

# Variant QC for step 1 via plink2

Use [plink2 to perform the variant QC](https://rgcgithub.github.io/regenie/recommendations/#exclusion-files) and obtain a subset of SNPs roughly equal to the number of samples.

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_MERGED_BGEN} ref-first \
  --sample {LOCAL_MERGED_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_PHENOTYPES} \
  --geno 0.1 \
  --mind 0.1 \
  --mac 100 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_step1QC_plink

# This is too strict and removes too many variants.
#  --maf 0.01 \

In [ ]:
!ls -lth . | head

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!tail {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step1QC_plink.id

In [ ]:
!head {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!tail {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step1QC_plink.snplist

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}* {OUTPUT_FOLDER}

# Variant QC for step 2 via plink2

In [ ]:
!/tmp/plink2/plink2 \
  --bgen {LOCAL_MERGED_BGEN} ref-first \
  --sample {LOCAL_MERGED_BGEN_SAMPLE} \
  --chr 1-22 \
  --keep {LOCAL_PHENOTYPES} \
  --geno 0.1 \
  --mind 0.1 \
  --hwe 1e-15 \
  --write-snplist \
  --write-samples \
  --no-id-header \
  --out {OUTPUT_FILENAME_PREFIX}_step2QC_plink

# This differs from step 1 QC in that the following parameter was removed
#    --mac 100 \

In [ ]:
!ls -lth {OUTPUT_FILENAME_PREFIX}*

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step2QC_plink.id

In [ ]:
!wc -l {OUTPUT_FILENAME_PREFIX}_step2QC_plink.snplist

In [ ]:
!gsutil -m cp {OUTPUT_FILENAME_PREFIX}* {OUTPUT_FOLDER}

# Provenance 

In [ ]:
%%bash

date

In [ ]:
%%bash

pip3 freeze